In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os,sys,time
from glob import glob
from scipy.optimize import fmin
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy import interpolate
import sympy as sp

In [2]:
def is_in_notebook():
    import sys
    return 'ipykernel' in sys.modules

def clear_output():
    """
    clear output for both jupyter notebook and the console
    """
    import os
    os.system('cls' if os.name == 'nt' else 'clear')
    if is_in_notebook():
        from IPython.display import clear_output as clear
        clear()
        
print('bbb')
clear_output()

In [3]:
def check_lc(datadir,lim=(0,0),error=False):
    data = np.genfromtxt(datadir,names="time,flux,et,ef")
    use = (data['ef'] == 0.)

    if error == True:
        time_diff = data['time'][use][1:] - data['time'][use][:-1]
        error_point = np.argmax(time_diff)
        Time = np.append(data['time'][use][10:error_point-10],data['time'][use][error_point+10:-10])
        flux = np.append(data['flux'][use][10:error_point-10],data['flux'][use][error_point+10:-10])
    else:
        Time = data['time'][use]
        flux = data['flux'][use]

    med = np.median(flux) 
    std = np.std(flux)
    w = (flux < (med - 3*std))
    

    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plt.plot(Time,flux,'c')
    plt.plot(Time,flux,'r.')
    plt.plot([Time[0],Time[-1]],[med-3*std,med-3*std],'g-')
    plt.xlabel('time');plt.ylabel('flux')
    if lim[0] != 0:
        plt.xlim(lim[0],lim[1])
    plt.grid()
    plt.subplot(122)
    plt.plot(Time,flux/med,'c')
    plt.plot(Time,flux/med,'r.')
    plt.plot([Time[0],Time[-1]],[1-3*std/med,1-3*std/med],'g-')
    plt.xlabel('time');plt.ylabel('flux/median')
    if lim[0] != 0:
        plt.xlim(lim[0],lim[1])
    plt.grid()
    plt.show()
    
    return len(flux[w])

In [64]:
def light_curve(lc_dir,look=False):
    data = np.genfromtxt(lc_dir,names="time,flux,et,ef")
    good = (data['ef'] == 0)
    time_diff = data['time'][good][1:] - data['time'][good][:-1]
    
    if (np.max(time_diff) > 2):
        return "bad_data","bad_data","bad_data"
    else:
        None
        
    error_point = np.argmax(time_diff)

    Time = np.append(data['time'][good][5:error_point-10],data['time'][good][error_point+10:-5])
    flux = np.append(data['flux'][good][5:error_point-10],data['flux'][good][error_point+10:-5])
    et = np.append(data['et'][good][5:error_point-10],data['et'][good][error_point+10:-5])
    
    min_flux = np.min(np.append(flux[int(len(flux)*0.05):int(len(flux)*0.4)],\
                                flux[int(len(flux)*0.6):-int(len(flux)*0.05)]))

    if lc_dir[-9:-7] == '11':
        w = (Time>3600) & (Time<3610)
        Time = Time[w]
        flux = flux[w]
        et = et[w]
        min_flux = np.min(flux)
        
    
    norm_flux = (flux - np.median(flux))/(np.median(flux) - min_flux)
    norm_et = et / np.abs(np.median(flux) - min_flux)

    if look == True:
        plt.figure(figsize=(10,5))
        plt.plot(Time,norm_flux,'c')
        plt.xlabel('time');plt.ylabel('flux')
        plt.show()
    else:
        None

    w = (norm_flux < 0.25) & (norm_flux > -1.2)
    return Time[w],norm_flux[w],norm_et[w]

def mulit_light_curve(lc_dir,lim=(0,0),look=False):
    Time = []; flux = []; et = []
    size = 0; sec_size = []
    for i in range(10):
        test_dir = lc_dir[:-9] + "0%s" %i + "_LC.txt"
        if os.path.exists(test_dir) == True:
            t,f,e = light_curve(test_dir)
            if t == 'bad_data':
                None
            else:
                Time.extend(t)
                flux.extend(f)
                et.extend(e)
                size += 1
                sec_size.append(i)
        else:
            None

    for i in range(10):
        test_dir = lc_dir[:-9] + "1%s" %i + "_LC.txt"
        if os.path.exists(test_dir) == True:
            t,f,e = light_curve(test_dir)
            if t == 'bad_data':
                None
            else:
                Time.extend(t)
                flux.extend(f)
                et.extend(e)
                size += 1
                sec_size.append(i+10)
        else:
            None
            
    if look == True:
        plt.figure(figsize=(5*(size+1),5))
        plt.title('SEC size = %s'%sec_size)
        plt.plot(Time,flux,'c')
        plt.plot(Time,flux,'r.')
        plt.xlabel('time');plt.ylabel('flux')
        if lim[0] != 0:
            plt.xlim(lim[0],lim[1])
        plt.grid()
        plt.show()
    else:
        None

    return Time, flux, et

In [65]:
def fit_peak(lc_dir,h=0.5,look=False):
    Time, flux, et = mulit_light_curve(lc_dir)
    Time = np.array(Time); flux = np.array(flux); et = np.array(et)
    
    peaks = find_peaks(-flux, height=h, distance=10)

    peaks_pos = peaks[0]
    height = -peaks[1]['peak_heights']
    
    need_del = []
    for i in range(len(Time[peaks_pos])):
        if ((peaks_pos + 5)[i] > len(Time)) or ((peaks_pos - 5)[i] < 0) :
            need_del.append(i)
            
    peaks_pos = np.delete(peaks_pos, need_del)
    height = np.delete(height, need_del)
    fittime = Time[peaks_pos]

    if look == True:
        plt.figure(figsize=(15,5))
        plt.subplot(121)
        plt.plot(Time,flux,'co-')
        plt.plot(fittime,height,'rx')
        plt.grid()

        plt.subplot(122)
        plt.plot(Time,flux,'co-')
        plt.plot(fittime,height,'rx')
        plt.xlim(fittime[0]-0.2,fittime[2]+0.2)
        plt.grid()
        plt.show()
    
    return np.array(fittime),np.array(height)

In [96]:
def Phase_cycle(t,P,T_0):
    T_diff = np.array(t) - T_0
    I = np.round(T_diff/P)
    phi = (T_diff/P) - I
    return phi, I #phase and cycle


def phase_average(x,y,bins=100):
    if bins >= len(x):
        return np.array(x),np.array(y)
    else:
        size = int(len(x)/bins)
        newx = []; newy = []
        for i in range(bins):
            newx.append(np.mean(x[i*size:(i+1)*size]))
            newy.append(np.mean(y[i*size:(i+1)*size]))
        newx.append(np.mean(x[bins*size:]))
        newy.append(np.mean(y[bins*size:]))
        return np.array(newx),np.array(newy)
    

def sing_dc_func(lc_dir,guess_P,look=False):
    Time, flux, et = mulit_light_curve(lc_dir)
    if len(Time) < 1:
        return None,None,None
    Time = np.array(Time); flux = np.array(flux); et = np.array(et)
    
    phi, I = Phase_cycle(Time, guess_P, Time[0])
    phase_flux = np.stack((phi,flux),axis=-1)
    phase_flux = np.array(sorted(phase_flux,key=lambda x:x[0]))
    
    phase = phase_flux[:,0]
    flux = phase_flux[:,1]

    mphase,mflux = phase_average(phase,flux,int(len(phase)/10))
    
    if look == True:
        plt.figure(figsize=(15,5))
        plt.title('%s -- period = %.6f'%(lc_dir[-28:-14],guess_P))
        plt.plot(phase,flux,'c-',alpha=0.5)
        plt.plot(phase,flux,'r.',alpha=0.2)
        plt.plot(mphase,mflux,'k-')
        plt.xlabel('phase');plt.ylabel('flux')
        plt.grid()
        plt.show()        
    
    S = np.fabs(flux[-1] - flux[0]) + np.sum(np.fabs(flux[1:] - flux[:-1]))
    return S/47.-2.,mphase,mflux

In [67]:
def guess_period(t,f): # eclipse time and flux
    d1 = np.abs(f[1]-f[0])
    d2 = np.abs(f[2]-f[0])
    if (d1 > 0.15) & (d2 < d1):
        period = t[2] - t[0]
    else:
        period = t[1] - t[0]
    return period


def hist_test(lc_dir,period=0,look=False):
    ecl_time, height = fit_peak(lc_dir,h=0.6)
    if len(ecl_time) < 2:
        return 0,0,0
    elif len(ecl_time) == 2:
        real_p = ecl_time[1] - ecl_time[0]
        ecc_period = 999
        return real_p, ecc_period, ecl_time[0]
    else:
        None
    
    if period == 0:
        period = guess_period(ecl_time,height)
    else:
        None
        
    T0 = ecl_time[0]
    bin_num = 50
    limit = 0.1*period
    ppp = np.linspace(period-limit,period+limit,5000)
    hist_weight = []
    for pp in ppp:
        phi, I = Phase_cycle(ecl_time,pp,T0)
        hist_info = np.histogram(phi,bins=bin_num)[0]
        hist_weight.append(np.max(hist_info))
        
    real_p = ppp[np.argmax(hist_weight)]
    
    if look == True:
        plt.figure(figsize = (10,5))
        plt.subplot(121)
        plt.title('period = %.6f' %real_p)
        plt.plot(ppp,hist_weight,'c')
        
        phi, I = Phase_cycle(ecl_time,real_p,T0)
        plt.subplot(122)
        plt.hist(phi,bins=bin_num)
        plt.show()
    
    phi, I2 = Phase_cycle(ecl_time,real_p,T0)
    sigy = np.sqrt(1/(len(ecl_time)-2)* np.sum((ecl_time-I2*real_p-T0)**2))
    Delta = len(ecl_time)*np.sum(ecl_time**2) - (np.sum(ecl_time))**2
    ecc_period = sigy * np.sqrt(len(ecl_time)/Delta)
    
    return real_p, ecc_period, ecl_time[0]

In [68]:
basedir = '/Users/laote/sdsu/Research/binary/curve_bin/'
base3wdir = basedir + '3w_binary/'
base16dir = basedir + '1600K_Curve_binary/'
checkdir = base16dir + 'tic00389920099_SEC007_LC.txt'


# binary_name_all = glob(base3wdir+'*.txt')
binary_name_all = np.genfromtxt('./curve_bin/binary_1.txt',dtype=str)
binary_name = []
for bbb in binary_name_all:
    if bbb[-28:-14] not in binary_name:
        binary_name.append(bbb[-28:-14])
print(len(binary_name))        

known_binary = np.genfromtxt(basedir+'binary_list.txt',dtype=str,skip_header=1)
known_tic_name = known_binary[:,0]

bbb = [];ccc = []
for a in binary_name_all:
    if a[-28:-14] not in known_tic_name:
        if a[-28:-14] not in ccc:
            bbb.append(base3wdir+a[-28:])
        ccc.append(a[-28:-14])
bbb = sorted(bbb, key=lambda name: int(name[-25:-14] + name[-10:-7]))
print(len(bbb))

4848
4439


In [117]:
if os.path.exists('period_3w_v1.npy'):
    period_v1 = np.load('period_3w_v1.npy')
    print("Success loading old file.")
else:
    period_v1 = []
period_v1 = np.array(period_v1)

for i in range(len(period_v1),4439):
#     print(bbb[i])
    a,b,c = mulit_light_curve(bbb[i],look=False)
    p,ep,t0 = hist_test(bbb[i])
    a,b,c = sing_dc_func(bbb[i],p,look=False)
    if (np.min(c) < -0.6) & (p != 0):
        bn = bbb[i][-28:-14]
        test = np.array([[bn, p, ep, t0]])
        if len(period_v1) != 0: #check empty
            if bn in period_v1[:,0]: #delete old data
                row_num = np.argwhere(period_v1[:,0] == bn)[0][0]
                period_v1 = np.delete(period_v1, row_num, 0)
            else:
                None

            period_v1 = np.vstack((test,period_v1))
            np.save('period_3w_v1',period_v1)
        else:
            period_v1 = test
            np.save('period_3w_v1',period_v1)
    else:
        None
    if i%50 == 0:
        clear_output()
    print(i)
    

4400
4401
4402
4403
4404
4405
4406
4407
4408
4409
4410
4411
4412
4413
4414
4415
4416
4417
4418
4419
4420
4421
4422
4423
4424
4425
4426
4427
4428
4429
4430
4431
4432
4433
4434
4435
4436
4437
4438


In [125]:
len(np.load('period_3w_v1.npy'))

2558